**This notebook is an exercise in the [Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/manipulating-geospatial-data).**

---


# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [1]:
import math
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim            # What you'd normally run
from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex4 import *

You'll use the `embed_map()` function from the previous exercise to visualize your maps.

이전 연습의 'embed_map()' 함수를 사용하여 지도를 시각화합니다.

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

다음 코드 셀을 실행하여 캘리포니아 주에 있는 Starbucks 위치를 포함하는 DataFrame 'starbucks'를 만듭니다.

In [4]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
starbucks.head()

Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

대부분의 상점은 (위도, 경도) 위치를 알고 있습니다. 그러나 버클리시의 모든 위치가 누락되었습니다.

In [5]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Use the code cell below to fill in these values with the Nominatim geocoder.

Note that in the tutorial, we used `Nominatim()` (from `geopy.geocoders`) to geocode values, and this is what you can use in your own projects outside of this course.  

In this exercise, you will use a slightly different function `Nominatim()` (from `learntools.geospatial.tools`).  This function was imported at the top of the notebook and works identically to the function from GeoPandas.

So, in other words, as long as: 
- you don't change the import statements at the top of the notebook, and 
- you call the geocoding function as `geocode()` in the code cell below, 

your code will work as intended!

-------------------------------------------------------------------------------------
아래 코드 셀을 사용하여 Nominatim 지오코더로 이 값을 채우십시오.

튜토리얼에서 우리는 값을 지오코딩하기 위해 `Nominatim()`(`geopy.geocoders`에서)을 사용했으며 이것은 이 과정 이외의 자신의 프로젝트에서 사용할 수 있는 것입니다.

이 연습에서는 약간 다른 함수 `Nominatim()`(`learntools.geospatial.tools`에서)을 사용합니다. 이 기능은 노트북 상단에서 가져온 것으로 GeoPandas의 기능과 동일하게 작동합니다.

즉,
- 노트북 상단의 import 문을 변경하지 않고,
- 아래 코드 셀에서 지오코딩 함수를 `geocode()`로 호출합니다.

코드가 의도한 대로 작동합니다!

In [12]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None
    
bk_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
bk_locations

# starbucks DataFrame에는 여전히 10개의 누락된 항목이 있습니다. 이 값을 입력하십시오.
# starbucks DataFrame에 bk_locations을 추가
starbucks.update(berkeley_locations)
starbucks

# missimg value에 값이 들어갔는지 확인
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

In [13]:
# Check your answer
q_1.check()

In [14]:
# Line below will give you solution code
#q_1.solution()

#> starbucks에 berkeley_locations의 값을 missimg_value에 업데이트한다.
#> starbucks.update(berkeley_locations)

### 2) View Berkeley locations.

Let's take a look at the locations you just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

방금 찾은 위치(버클리 위치)를 살펴보겠습니다. OpenStreetMap 스타일로 버클리의 (위도, 경도) 위치를 시각화합니다.

In [17]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
df = starbucks[starbucks["City"]=='Berkeley']

for idx, row in df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Store Name']).add_to(m_2)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m_2, 'q_2.html')

In [18]:
# Get credit for your work after you have created a map
q_2.a.check()

# Uncomment to see our solution (your code may look different!)
#q_2.a.solution()

Considering only the five locations in Berkeley, how many of the (latitude, longitude) locations seem potentially correct (are located in the correct city)?

버클리의 5개 위치만 고려하면, (위도, 경도) 위치 중 몇 개(위도, 경도)가 잠재적으로 올바른 것처럼 보입니까(올바른 도시에 위치)?

A. Solano Ave, Safeway-Berkeley#691, 2224 Shattuck-Berkeley, 2128 Oxford St.,Telegraph & Ashby 5개가 올바른 위치에 있어 보인다.

In [19]:
# View the solution (Run this code cell to receive credit!)
q_2.b.solution()

### 3) Consolidate your data.

Run the code below to load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

아래 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱 킬로미터) 및 고유 ID("GEOID" 열에 있음)가 포함된 GeoDataFrame `CA_counties`를 로드합니다. "형상" 열에는 카운티 경계가 있는 다각형이 포함되어 있습니다.

In [20]:
CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

--------------------------------------------------
다음으로 3개의 DataFrame을 생성합니다.
- 'CA_pop'은 각 카운티의 인구 추정치를 포함합니다.
- `CA_high_earners`에는 연간 소득이 $150,000 이상인 가구 수가 포함됩니다.
- 'CA_median_age'는 각 카운티의 중위 연령을 포함합니다.

In [31]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

Use the next code cell to join the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`.

Name the resultant GeoDataFrame `CA_stats`, and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  Also, make sure the CRS is set to `{'init': 'epsg:4326'}`.

--------------------------------------------------
다음 코드 셀을 사용하여 `CA_counties` GeoDataFrame을 `CA_pop`, `CA_high_earners` 및 `CA_median_age`와 결합합니다.

결과 GeoDataFrame의 이름을 `CA_stats`로 지정하고 "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners" 및 "median_age"의 8개 열이 있는지 확인합니다. 또한 CRS가 `{'init': 'epsg:4326'}`으로 설정되어 있는지 확인하십시오.

In [29]:
print(CA_pop.head(),
      "\n\n",
      CA_high_earners.head(),
      "\n\n",
      CA_median_age.head())

In [35]:
# Your code here
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

CA_stats.crs = {'init': 'epsg:4326'}
CA_stats.head()

In [36]:
# Check your answer
q_3.check()

In [37]:
# Lines below will give you a hint or solution code
q_3.hint()
#> pd.DataFrame.join()을 사용하여 3개의 DataFrame을 함께 결합하는 것으로 시작합니다. 
#> 그런 다음 이 새 DataFrame을 gpd.GeoDataFrame.merge()를 사용하여 CA_countiesGeoDataFrame에 추가합니다.

#q_3.solution()

Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  Run the next code cell to create a "density" column with the population density.

이제 모든 데이터가 한 곳에 있으므로 열 조합을 사용하는 통계를 훨씬 쉽게 계산할 수 있습니다. 다음 코드 셀을 실행하여 인구 밀도가 있는 "밀도" 열을 만듭니다.

In [40]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats.head()

### 4) Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

------------------------------------------------------------------
모든 정보를 단일 GeoDataFrame으로 축소하면 특정 기준을 충족하는 카운티를 훨씬 더 쉽게 선택할 수 있습니다.

다음 코드 셀을 사용하여 `CA_stats` GeoDataFrame에서 행(및 모든 열)의 하위 집합을 포함하는 GeoDataFrame `sel_counties`를 만듭니다. 특히 다음과 같은 카운티를 선택해야 합니다.
- 연간 소득이 15만 달러 이상인 가구가 10만 가구 이상인 경우
- 중위 연령이 38.5세 미만이고,
- 주민 밀도는 최소 285명(제곱 킬로미터당)입니다.

또한 선택한 카운티는 다음 기준 중 하나 이상을 충족해야 합니다.
- 연간 $150,000를 버는 최소 500,000 가구가 있고,
- 중위 연령이 35.5세 미만이거나
- 주민 밀도는 최소 1400명(제곱 킬로미터당)입니다.

In [43]:
# Your code here
# CA_stats GeoDataFrame에서 행(및 모든 열)의 하위 집합을 포함하는 GeoDataFrame sel_counties를 만듭니다.
sel_counties = CA_stats[(
                         # 다음과 같은 카운티를 선택
                         (CA_stats.high_earners > 100000) & # 연간 소득이 15만 달러 이상인 가구가 10만 가구 이상인 경우
                         (CA_stats.median_age < 38.5) & # 중위 연령이 38.5세 미만
                         (CA_stats.density > 285) & # 주민 밀도는 최소 285명
                         
                         # 기준 중 하나 이상을 충족
                         ((CA_stats.median_age < 35.5) | # 연간 $150,000를 버는 최소 500,000 가구
                         (CA_stats.density > 1400) | # 중위 연령이 35.5세 미만
                         (CA_stats.high_earners > 500000)))] # 주민 밀도는 최소 1400명

sel_counties

In [44]:
# Check your answer
q_4.check()

In [45]:
# Lines below will give you a hint or solution code
q_4.hint()
#q_4.solution()

### 5) How many stores did you identify?

When looking for the next Starbucks Reserve Roastery location, you'd like to consider all of the stores within the counties that you selected.  So, how many stores are within the selected counties?

To prepare to answer this question, run the next code cell to create a GeoDataFrame `starbucks_gdf` with all of the starbucks locations.

-------------------------------------
다음 Starbucks Reserve Roastery 위치를 찾을 때 선택한 카운티 내의 모든 매장을 고려하고 싶습니다. 그렇다면 선택한 카운티 내에 몇 개의 매장이 있습니까?

이 질문에 답할 준비를 하려면 다음 코드 셀을 실행하여 모든 스타벅스 위치가 포함된 GeoDataFrame `starbucks_gdf`를 만듭니다.

In [47]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}
starbucks_gdf

So, how many stores are in the counties you selected?

그렇다면 선택한 카운티에는 몇 개의 매장이 있습니까?

A. 1043개의 매장이 있습니다.

In [50]:
# Fill in your answer
df_sjoin = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(df_sjoin)
num_stores

In [51]:
# Check your answer
q_5.check()

In [49]:
# Lines below will give you a hint or solution code
q_5.hint()
#q_5.solution()

### 6) Visualize the store locations.

Create a map that shows the locations of the stores that you identified in the previous question.

이전 질문에서 식별한 상점의 위치를 보여주는 지도를 만드십시오.

In [54]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
mc = MarkerCluster()

df_sjoin = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in df_sjoin.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Uncomment to see a hint
q_6.hint()

# Show the map
embed_map(m_6, 'q_6.html')

In [55]:
# Get credit for your work after you have created a map
q_6.check()

# Uncomment to see our solution (your code may look different!)
q_6.solution()

# Keep going

Learn about how **[proximity analysis](https://www.kaggle.com/alexisbcook/proximity-analysis)** can help you to understand the relationships between points on a map.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*